<h1> Programowanie w QGIS - modyfikacje warstw </h1>

Na ostatnich ćwiczeniach ładowaliśmy warstwy, dokonywaliśmy selekcji i uzyskiwaliśmy dostęp do wartości używając klas warstw wektorowych, rastrowych, projektu QGIS oraz dataProvider przekazującego dane. Możliwe jest również wygodne edytowanie parametrów i wartości tych obiektów za pomocą poleceń Pythona.

In [ ]:
#obecnie zaznaczona warstwa
warstwa = iface.activeLayer()

In [ ]:
#odświeżanie
warstwa.triggerRepaint()
#LUB
iface.mapCanvas().refresh()

<h3> Dodawanie nowych obiektów </h3>

Każdy obiekt warstwy wektorowej ma:
<ul>
    <li> Atrybuty </li>
    <li> Geometrię </li>
</ul>

Przy tworzeniu obiektów należy ustalić oba.

Nowe klasy!!

1. QgsFeature - klasa obiektu warstwy wektorowej

2. QgsField - klasa pola wartości w atrybutach warstwy wektorowej

3. QgsGeometry - klasa geometrii obiektu

In [ ]:
feat = QgsFeature(warstwa.fields()) #instancja obiektu QgsFeature, parametrem są pola wartości QgsField warstwy

In [ ]:
feat.setAttributes([0, 60]) #dodajemy atrybuty
#LUB
feat.setAttribute(0,60) #tylko jeden wiersz

In [ ]:
#Tworzymy geometrię - linia z dwóch punktów
line_start = QgsPoint(15.74,50.89)
line_end = QgsPoint(15.73,50.91)
feat.setGeometry(QgsGeometry.fromPolyline([line_start,line_end]))

QgsGeometry to klasa geometrii, która może przyjmować postać punktów, linii i poligonów. Najczęściej budujemy jest z punktów, czyli z klasy ([nowa klasa!]) QgsPoint:

    QgsPoint(x,y)

Następnie stosujemy jedno z poleceń .fromGeometria, czyli:

    fromPolyline, fromPolygonXY, fromPointXY lub fromWKT - czyli typu danych WKT

In [ ]:
warstwa.dataProvider().addFeatures([feat]) #dodajemy obiekt do warstwy

<h3> WKT - Well-known text representation of geometry </h3>

To jeden ze standardów OGC, który pozwala na opisanie każdego typu geometrii:

    Point, MultiPoint
    LineString, MultiLineString,
    Polygon, MultiPolygon

Definicja:

    Nazwa (x y)

Punkt:

    POINT (10 30)

Linia:

    LINESTRING (30 10, 10 30, 40 40)
    
Poligon:

    POLYGON ((30 10, 40 40, 20 20, 10 20, 30 10))
    
    POLYGON ((35 10, 45 45, 15 40, 10 20, 35 10), (20 30, 35 35, 30 20, 20 30))
    
    

<h3> WKB - Well-known binary </h3>

Binarny odpowiednik WKT

<h3> Usuwanie obiektów </h3>

Wystarczy podać ich indeks:

In [ ]:
warstwa.dataProvider().deleteFeatures([0])

<h3> Edycja obiektów </h3>

Możemy edytować atrybuty lub geometrię. Pozwalają na to metody klasy dataProvider


In [ ]:
attrs = { 0 : 100, 1 : 90 }
warstwa.dataProvider().changeAttributeValues({ 0 : attrs })

In [ ]:
geom = QgsGeometry.fromPolyline([QgsPoint(15.70,50.59),QgsPoint(15.70,50.52),QgsPoint(15.73,50.52)])
warstwa.dataProvider().changeGeometryValues({0: geom})

<h3> Ostrożności nigdy za wiele - tryb edycji </h3>

Zmiany dokonane w trybie edycji mogą zostać odwrócone, dlatego też przed każdą decyzją najlepiej skorzystać jest z metod warstwy wektorowej:

    startEditing() - rozpocznij edycje
    commitChanges() - dokonaj zmian
    rollBack() - przywróć zmiany
    
    isEditable() - sprawdź czy warstwa jest edytowalna
    


<h3> Nowe atrybuty </h3>

dataProvider() posiada metodę addAttributes oraz deleteAttributes.

[Nowa klasa] - QVariant, klasa przenosząca typ danych

In [ ]:
warstwa.dataProvider().addAttributes\
([QgsField("nazwa_string", QVariant.String), QgsField("nazwa_int", QVariant.Int)])

In [ ]:
warstwa.dataProvider().deleteAttributes([0])

In [ ]:
warstwa.updateFields() #BARDZO WAŻNE, zawsze należy potwierdzić operację

<h3> Tworzenie warstw, zapisywanie </h3>

Zapisywanie warstw wektorowych obsługuje inna klasa - $QgsVectorFileWriter$ i jej funkcja $writeAsVectorFormat()$, która wspiera wszystkie formaty danych OGR (GeoPackage, Shapefile, GeoJSON, KML i inne)

In [ ]:
QgsVectorFileWriter.writeAsVectorFormat(warstwa, "nazwa", "UTF-8") #(warstwa, nazwa, kodowanie znaków)

In [ ]:
QgsVectorFileWriter.writeAsVectorFormat(warstwa, "nazwa", "UTF-8", driverName="GeoJSON")

In [ ]:
crs=QgsCoordinateReferenceSystem("epsg:4326")
QgsVecotFileWriter.writeAsVectorFormat(warswa, "nazwa", "UTF-8", crs = crs, driverName="GeoJSON")

<h3> Warstwy tymczasowe </h3>

W QGIS2 warstwy tymaczasowe nie były w ogóle dopuszczalne w aplikacji QGISa. W QGIS3 są, lecz nie zawsze. Należy pamiętać że nie zawsze nadają się do geoprocessingu. Pozwalają oszczędzić pamięć i miejsce na dysku twardym podczas operacji.

In [ ]:
from qgis.PyQt.QtCore import QVariant

#pusta warstwa w pamięci
temp_vlayer = QgsVectorLayer("Point", "tymczasowe_punkty", "memory")
pr = temp_vlayer.dataProvider()

#Atrybuty - pola
pr.addAttributes([QgsField("nazwa", QVariant.String), QgsField("wiek",QVariant.Int)])
temp_vlayer.updateFields()

#obiekt
feat = QgsFeature() #pusty obiekt
#Geometria
feat.setGeometry(QgsGeometry.fromPointXY(QgsPointXY(10,10)))
#Atrybuty
feat.setAttributes(["Bobby",12])
#Dodaj
pr.addFeatures([feat])
temp_vlayer.updateExtents() #odśwież warstwę